# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part C | Scrapping and Amazon Product Reviews (cont.)

> ## We are now ready to extract the reviews offline and no longer need to query the Amazon website.

In [1]:
import os
import gzip
import json
import lxml.html
import dateutil

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

> ## Input

In [2]:
with gzip.open(os.path.join('..', 'datasets', 'dataset-04-reviews.json.gz'), 'rb') as f:
    pages = json.loads(f.read())

In [3]:
len(pages)

2

## First page

In [4]:
page = pages['1']['content']

In [6]:
document = lxml.html.fromstring(page)

In [7]:
type(document)

lxml.html.HtmlElement

(http://lxml.de/api/lxml.html-module.html#fromstring and http://lxml.de/api/lxml.html.HtmlElement-class.html)

> ## All reviews of a page

(http://lxml.de/api/lxml.etree._Element-class.html#xpath)

In [8]:
reviews = document.xpath('//*[@data-hook="review"]')

## First review

In [9]:
review = reviews[0]

In [10]:
type(review)

lxml.html.HtmlElement

> ## id

(http://lxml.de/api/lxml.etree._Element-class.html#get)

In [11]:
review.get('id')

'R29ISC3WOB5X8E'

> # star rating

In [12]:
review.find('.//*[@data-hook="review-star-rating"]').get('class')

'a-icon a-icon-star a-star-5 review-rating'

(http://lxml.de/api/lxml.etree._Element-class.html#find)

(https://en.wikipedia.org/wiki/XPath)

> ## title

In [13]:
review.findtext('.//*[@data-hook="review-title"]')

'Highly recommend'

(http://lxml.de/api/lxml.etree._Element-class.html#findtext)

> ## author

In [14]:
review.findtext('.//*[@data-hook="review-author"]/*[@data-hook="review-author"]')

'Mason selby'

> ## date

In [ ]:
# TODO

> ## body

In [ ]:
# TODO

> ## Output

In [ ]:
df = pd.DataFrame(columns = ['date', 'id', 'author', 'title', 'body', 'star_rating'])

In [ ]:
df

## Putting all of it together

(https://docs.python.org/2/howto/unicode.html and https://docs.python.org/2/library/stdtypes.html)

In [ ]:
def date(node):
    date = review.findtext('.//*[@data-hook="review-date"]').replace('on ', '')
    return dateutil.parser.parse(date)

def id(node):
    return node.get('id')

def author(node):
    return node.findtext('.//*[@data-hook="review-author"]/*[@data-hook="review-author"]').encode('ascii', 'ignore')

def title(node):
    return node.findtext('.//*[@data-hook="review-title"]').encode('ascii', 'ignore')

def body(node):
    return node.findtext('.//*[@data-hook="review-body"]').encode('ascii', 'ignore')

def star_rating(node):
    node = node.find('.//*[@data-hook="review-star-rating"]')

    if node == None:
        return np.nan

    for star_rating in range(1, 6):
        if node.find_class('a-star-{:d}'.format(star_rating)):
             return star_rating

    return np.nan

(http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html)

In [ ]:
for i in sorted(pages.keys(), key = lambda i: int(i)):
    page = pages[i]['content']
    document = lxml.html.fromstring(page)
    reviews = document.xpath('//*[@data-hook="review"]')

    for review in reviews:
        df = df.append({'date': date(review),
                        'id': id(review),
                        'author': author(review),
                        'title': title(review),
                        'body': body(review),
                        'star_rating': star_rating(review)},
                       ignore_index = True)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.to_csv(os.path.join('..', 'datasets', 'dataset-04-reviews.csv'), index = False)